In [19]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [20]:
loc = '/p/scratch/exaww/chatterjee1/msg_netcdf/2023/07/'
file = 'HRSEVIRI_20230731T114510Z_20230731T115742Z_epct_92f37c8d_PC.nc'
save_path = '/p/scratch/exaww/chatterjee1/msg_netcdf/images/'

In [21]:
C1 = 1.19104*10**(-5)  # in [mW (cm−1)−4 m-2 sr−1]
C2 = 1.43877  # in [K cm]

CHANNEL_NAME = {"channel_1": "VIS 0.6", 
                "channel_2": "VIS 0.8", 
                "channel_3": "NIR 1.6", 
                "channel_4": "IR 3.9", 
                "channel_5": "WV 6.2", 
                "channel_6": "WV 7.3", 
                "channel_7": "IR 8.7", 
                "channel_8": "IR 9.7 - O3", 
                "channel_9": "IR 10.8", 
                "channel_10": "IR 12.0", 
                "channel_11": "IR 13.4 - CO2", }
# in [cm−1]
VC = {'MSG1': {"channel_4": 2567.330, "channel_5": 1598.103, "channel_6": 1362.081, "channel_7": 1149.069, 
                "channel_8": 1034.343, "channel_9": 930.647, "channel_10": 839.660, "channel_11": 752.387
                }, 
      'MSG2': {"channel_4": 2568.832, "channel_5": 1600.548, "channel_6": 1360.330, "channel_7": 1148.620, 
                "channel_8": 1035.289, "channel_9": 931.700, "channel_10": 836.445, "channel_11": 751.792
                }, 
      'MSG3': {"channel_4": 2547.771, "channel_5": 1595.621, "channel_6": 1360.377, "channel_7": 1148.130, 
                "channel_8": 1034.715, "channel_9": 929.842, "channel_10": 838.659, "channel_11": 750.653
                }, 
      'MSG4': {"channel_4": 2555.280, "channel_5": 1596.080, "channel_6": 1361.748, "channel_7": 1147.433, 
                "channel_8": 1034.851, "channel_9": 931.122, "channel_10": 839.113, "channel_11": 748.585
                }, }
# unitless
ALPHA = {'MSG1': {"channel_4": 0.9956, "channel_5": 0.9962, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, 
         'MSG2': {"channel_4": 0.9954, "channel_5": 0.9963, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, 
         'MSG3': {"channel_4": 0.9915, "channel_5": 0.9960, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9982
                   }, 
         'MSG4': {"channel_4": 0.9916, "channel_5": 0.9959, "channel_6": 0.9990, "channel_7": 0.9996, 
                   "channel_8": 0.9998, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, }
# in [K]
BETA = {'MSG1': {"channel_4": 3.410, "channel_5": 2.218, "channel_6": 0.478, "channel_7": 0.179, ''
                  "channel_8": 0.060, "channel_9": 0.625, "channel_10": 0.397, "channel_11": 0.578
                  },
        'MSG2': {"channel_4": 3.438, "channel_5": 2.185, "channel_6": 0.470, "channel_7": 0.179, 
                  "channel_8": 0.056, "channel_9": 0.640, "channel_10": 0.408, "channel_11": 0.561
                  },
        'MSG3': {"channel_4": 2.9002, "channel_5": 2.0337, "channel_6": 0.4340, "channel_7": 0.1714, 
                  "channel_8": 0.0527, "channel_9": 0.6084, "channel_10": 0.3882, "channel_11": 0.5390
                  },
        'MSG4': {"channel_4": 2.9438, "channel_5": 2.0780, "channel_6": 0.4929, "channel_7": 0.1731, 
                  "channel_8": 0.0597, "channel_9": 0.6256, "channel_10": 0.4002, "channel_11": 0.5635
                  }, }

# %%
#############
############# look up tables for calculating reflectances
#############
# constants taken from website: 
# https://eumetsatspace.atlassian.net/wiki/spaces/DSDT/pages/1537277953/MSG15+radiances+conversion+to+BT+and+Reflectances
# and from https://www-cdn.eumetsat.int/files/2020-04/pdf_msg_seviri_rad2refl.pdf

IRRAD = {'MSG1': {"channel_1": 65.2296, "channel_2": 73.0127, "channel_3": 62.3715},
         'MSG2': {"channel_1": 65.2065, "channel_2": 73.1869, "channel_3": 61.9923},
         'MSG3': {"channel_1": 65.5148, "channel_2": 73.1807, "channel_3": 62.0208}, 
         'MSG4': {"channel_1": 65.2656, "channel_2": 73.1692, "channel_3": 61.9416}, }


# %%
class ir_channel:
    """
    class that calls channel specific constants from look up tables above
    """
    def __init__(self, satellite, channel):

        self.name = CHANNEL_NAME[channel]
        self.vc = VC[satellite][channel]  # wavenumber in [cm−1]
        self.alpha = ALPHA[satellite][channel]  # unitless
        self.beta = BETA[satellite][channel]  # in [K]

class vis_nir_channel:
    def __init__(self, satellite, channel):
        
        self.name = CHANNEL_NAME[channel]
        self.irrad = IRRAD[satellite][channel]  # irradiance at 1AU in [mW·m-2·(cm-1)-1]

class MSG_satellite:
    def __init__(self, name):
        self.name =  name

    def _get_channel(self, channel_number):
        # return vis/nir or ir channel depending on channel number
        if channel_number <=3:
            return vis_nir_channel(satellite=self.name, channel=f"channel_{channel_number}")
        else:
            return ir_channel(satellite=self.name, channel=f"channel_{channel_number}")

    def rad_2_tb(self, channel_number, radiances):
        # error handling here:
        # TODO: raise exception when given incorrect channel_number, must be >=4

        # get constants for given channel
        channel_consts = self._get_channel(channel_number)

        # converting radiance to brightness temperature [K] with simplified equation
        numerator = C2 * channel_consts.vc
        fraction = C1 * channel_consts.vc**3 / radiances + 1
        denominator = channel_consts.alpha * (np.log(fraction))
        tb = numerator / denominator - channel_consts.beta / channel_consts.alpha  ## [K]
        return tb
    
    def _d(t):
        # Sun-Earth distance in AU at time t
        return None
    
    def _solar_zenith_angle(t, lon, lat):
        # Solar Zenith Angle in Radians at time t and location x
        return None

    def rad_2_refl(self, channel_number, radiances, t, lon, lat):
        # error handling here:
        # TODO: raise exception when given incorrect channel_number, must be <= 3

        # get constants for given channel
        channel_consts = self._get_channel(channel_number)

        numerator = np.pi * radiances * self._d(t)**2
        denominator = channel_consts.irrad * np.cos(self._solar_zenith_angle(t, lon, lat))

# %%
def radiances_2_brightnesstemp_and_reflectances(radiances, channel_number, satellite_name):
    ## radiances in [mW m−2 sr−1 (cm−1)−1)]
    # TODO: add constraint to channel_number (must be >= 4)

    # access correct satellite 
    satellite = MSG_satellite(satellite_name)
    if channel_number <= 3:
        print("not implemented yet for visible and near-infrared")

    elif channel_number >= 4 and channel_number < 12:
        # get brightness temp fro given channel
        return satellite.rad_2_tb(channel_number, radiances)
    
    else:
        print(f"This channel does not exist for satellite {satellite_name}")
        # TODO: raise exception

In [22]:
example_MSG = loc+file
with xr.open_dataset(example_MSG) as dataset:
    data = dataset
    print(data)

<xarray.Dataset>
Dimensions:     (lat: 665, lon: 958)
Coordinates:
  * lat         (lat) float64 30.02 30.06 30.09 30.13 ... 54.91 54.94 54.98
  * lon         (lon) float64 -3.481 -3.444 -3.406 -3.368 ... 32.41 32.44 32.48
Data variables:
    crs         |S1 ...
    channel_1   (lat, lon) float32 ...
    channel_2   (lat, lon) float32 ...
    channel_3   (lat, lon) float32 ...
    channel_4   (lat, lon) float32 ...
    channel_5   (lat, lon) float32 ...
    channel_6   (lat, lon) float32 ...
    channel_7   (lat, lon) float32 ...
    channel_8   (lat, lon) float32 ...
    channel_9   (lat, lon) float32 ...
    channel_10  (lat, lon) float32 ...
    channel_11  (lat, lon) float32 ...
Attributes: (12/45)
    Conventions:                    CF-1.5
    ch01_cal:                       -1.114656032994e+00 2.185600064695e-02
    ch02_cal:                       -1.465775717050e+00 2.874070033431e-02
    ch03_cal:                       -1.211260244250e+00 2.375020086765e-02
    ch04_cal:       

In [23]:
satellite_name = data.EPCT_product_name.split('-')[0]
print(satellite_name)
timestamp = data.EPCT_product_name.split('A-')[1].split('.')[0]
print(timestamp)

MSG3
20230731115742


In [10]:
data.EPCT_product_name.split('A-')[1].split('.')[0]

'20230731115742'

In [24]:
tb.shape

(665, 958)

In [42]:
for ch in np.arange(9, 10, 1):
    radiances = data[f"channel_{ch}"].values
    print(radiances)
    tb = radiances_2_brightnesstemp_and_reflectances(radiances, ch, satellite_name)
    print(tb)

[[       nan 155.41704  156.23718  ...        nan        nan        nan]
 [       nan 153.9818   154.80194  ... 144.34512  142.29475  140.2444  ]
 [       nan 153.16165  153.9818   ... 145.57533  143.31993  140.44943 ]
 ...
 [ 34.035923  33.830887  34.856064 ...  44.902813  44.902813  38.34167 ]
 [ 39.161816  39.161816  39.77692  ...  58.845238  47.978348  47.978348]
 [ 39.161816  39.161816  39.77692  ...  52.899204  58.845238  47.978348]]
[[      nan 323.32776 323.7338  ...       nan       nan       nan]
 [      nan 322.6144  323.02252 ... 317.72836 316.66605 315.59528]
 [      nan 322.20517 322.6144  ... 318.36182 317.19824 315.7027 ]
 ...
 [236.86859 236.61548 237.87126 ... 249.07866 249.07866 241.96999]
 [242.89915 242.89915 243.58788 ... 262.2514  252.18398 252.18398]
 [242.89915 242.89915 243.58788 ... 256.90204 262.2514  252.18398]]


In [35]:
radiances.shape, tb.shape

((665, 958), (665, 958))

In [26]:


# Assume your lat, long, and bt arrays are already loaded
lat = data.lat.values  # shape: (657,)
long = data.lon.values  # shape: (1377,)
bt_data = tb  # shape: (1, 657, 1377)

# Extract the first layer of 'bt'
#bt_data = bt[0, :, :]  # shape: (657, 1377)

# Create a meshgrid for lat and long
lon_grid, lat_grid = np.meshgrid(long, lat)

# Create the plot
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add coastlines for context
ax.coastlines()

# Plot the data
plt.pcolormesh(lon_grid, lat_grid, bt_data, cmap='viridis', shading='auto', transform=ccrs.PlateCarree())

# Add a colorbar
plt.colorbar(label='BT 10.8')

# Set labels and title
#plt.title('Parameter of Interest (bt) Projected in Lat-Lon Space')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Save the plot to a specific location
save_path_ = save_path + timestamp + '.png'
plt.savefig(save_path_, dpi=300, bbox_inches='tight')  # dpi=300 for high resolution, bbox_inches='tight' to trim whitespace

# Optionally, you can still show the plot if needed
plt.show()


## trying for central european domain

In [41]:
for ch in np.arange(8, 10, 1):
    radiances = data[f"channel_{ch}"][465:611,252:572].values
    print(radiances)
    tb = radiances_2_brightnesstemp_and_reflectances(radiances, ch, satellite_name)
    print(tb)
    print(ch)

[[38.77742  40.336834 41.376442 ... 38.257614 38.77742  38.361576]
 [38.361576 40.12891  40.232872 ... 39.609108 40.232872 38.98534 ]
 [33.995216 37.218006 38.465538 ... 37.945732 37.945732 36.906124]
 ...
 [23.599127 23.599127 23.911009 ... 30.98035  32.331844 32.331844]
 [23.495165 22.97536  23.287245 ... 30.98035  30.98035  32.019962]
 [22.97536  23.079323 22.97536  ... 29.83678  30.98035  32.019962]]
[[255.21169 256.94373 258.074   ... 254.62415 255.21169 254.74208]
 [254.74208 256.71536 256.82965 ... 256.14105 256.82965 255.44525]
 [249.5937  253.43298 254.85979 ... 254.26907 254.26907 253.07137]
 ...
 [235.22495 235.22495 235.71323 ... 245.77518 247.51639 247.51639]
 [235.06119 234.23494 234.73222 ... 245.77518 245.77518 247.11899]
 [234.23494 234.40121 234.23494 ... 244.26147 245.77518 247.11899]]
8
[[80.16895  84.67973  87.3452   ... 83.65456  84.88477  84.26966 ]
 [79.14377  83.244484 82.83441  ... 89.19052  88.98548  84.4747  ]
 [68.071846 75.45313  79.14377  ... 81.19413  81

In [38]:
radiances.shape, tb.shape

((146, 320), (146, 320))

In [43]:
# Assume your lat, long, and bt arrays are already loaded
lat = data.lat[465:611].values  # shape: (657,)
long = data.lon[252:823].values  # shape: (1377,)
radiances = data["channel_9"][465:611,252:823].values
bt_data = radiances_2_brightnesstemp_and_reflectances(radiances, ch, satellite_name)

# Create a meshgrid for lat and long
lon_grid, lat_grid = np.meshgrid(long, lat)

# Create the plot
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add coastlines for context
ax.coastlines()

# Plot the data
plt.pcolormesh(lon_grid, lat_grid, bt_data, cmap='viridis', shading='auto', transform=ccrs.PlateCarree())

# Add a colorbar
plt.colorbar(label='BT 10.8')

# Set labels and title
#plt.title('Parameter of Interest (bt) Projected in Lat-Lon Space')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Save the plot to a specific location
save_path_ = save_path + timestamp + '_CE' + '.png'
plt.savefig(save_path_, dpi=300, bbox_inches='tight')  # dpi=300 for high resolution, bbox_inches='tight' to trim whitespace

# Optionally, you can still show the plot if needed
plt.show()

In [44]:
bt_data.shape

(146, 571)

In [45]:
128*4

512

In [76]:
ds_ = xr.open_dataset('/p/scratch/exaww/chatterjee1/msg_netcdf/2023/' + 'msgobs_108_randcrops.nc')
ds_

<xarray.Dataset>
Dimensions:      (sample: 74268, y: 128, x: 128)
Coordinates:
  * sample       (sample) int64 0 1 2 3 4 5 ... 74263 74264 74265 74266 74267
    lat          (sample, y) float64 ...
    lon          (sample, x) float64 ...
    time         (sample) object ...
Dimensions without coordinates: y, x
Data variables:
    sample_data  (sample, y, x) float32 ...

In [69]:
ds_.sample_data.shape

(100, 128, 128)

In [67]:
ds_.sample_data[0,:,:].values

array([[238.36684, 239.83156, 240.55196, ..., 242.43614, 243.81589,
        243.81589],
       [243.58788, 245.1687 , 244.94505, ..., 246.49591, 245.61388,
        246.27644],
       [249.50032, 251.16322, 251.16322, ..., 251.3685 , 249.91957,
        248.65456],
       ...,
       [255.94254, 252.58844, 252.58844, ..., 245.83542, 245.1687 ,
        245.61388],
       [263.1394 , 263.1394 , 261.17307, ..., 240.3127 , 240.55196,
        241.96999],
       [263.492  , 263.1394 , 261.17307, ..., 240.3127 , 240.55196,
        240.55196]], dtype=float32)

In [58]:
ds_.lat.shape, ds_.lon.shape, ds_.time.shape

((72, 128), (72, 128), (72,))

In [53]:
ds = xr.open_dataset('/p/scratch/deepacf/kiste/DC/dataset/bigdata_centraleurope/nils/retro_atmos/samples/samples_352_nya.nc')
ds

<xarray.Dataset>
Dimensions:  (profile: 352, height: 200, sample: 71917)
Coordinates:
  * profile  (profile) int64 0 1 2 3 4 5 6 7 ... 344 345 346 347 348 349 350 351
  * height   (height) int64 100 150 200 250 300 ... 9850 9900 9950 10000 10050
  * sample   (sample) int64 0 1 2 3 4 5 ... 71911 71912 71913 71914 71915 71916
Data variables:
    ze       (sample, height, profile) float32 ...
    vm       (sample, height, profile) float32 ...
    beta     (sample, height, profile) float32 ...
    tb       (sample, height, profile) float32 ...
    time     (sample, profile) datetime64[ns] ...

In [54]:
ds.ze.shape

(71917, 200, 352)